<a href="https://colab.research.google.com/github/HakureiPOI/Comp_Arch/blob/lab4-5/Arch_Lab_4_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***HakureiPOI*** *Arch_Lab_4-5*

---
## 环境准备

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!nvidia-smi

Wed Dec 25 06:12:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!git clone https://github.com/HakureiPOI/Comp_Arch.git

Cloning into 'Comp_Arch'...
remote: Enumerating objects: 810, done.
remote: Counting objects: 100% (810/810), done.
remote: Compressing objects: 100% (439/439), done.
remote: Total 810 (delta 499), reused 605 (delta 321), pack-reused 0 (from 0)
Receiving objects: 100% (810/810), 7.46 MiB | 5.88 MiB/s, done.
Resolving deltas: 100% (499/499), done.


In [4]:
%cd Comp_Arch

/content/Comp_Arch


In [5]:
!git branch -a

* main
  remotes/origin/HEAD -> origin/main
  remotes/origin/lab1
  remotes/origin/lab2
  remotes/origin/lab3
  remotes/origin/lab4-5
  remotes/origin/main


In [6]:
!git switch lab4-5

Branch 'lab4-5' set up to track remote branch 'lab4-5' from 'origin'.
Switched to a new branch 'lab4-5'


In [7]:
%cd lab4-5

/content/Comp_Arch/lab4-5


---
## Lab4

### 编译

In [8]:
!nvcc -arch=compute_75 -L/usr/local/cuda/lib64 -lcublas ./matrix_mul_lab4.cu

./matrix_mul_lab4.cu(198): warning #177-D: variable "alpha" was declared but never referenced
    const float alpha = 1.0f;
                ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

./matrix_mul_lab4.cu(199): warning #177-D: variable "beta" was declared but never referenced
    const float beta = 0.0f;
                ^

./matrix_mul_lab4.cu(18): warning #177-D: variable "BLOCK_SIZE" was declared but never referenced
  const int BLOCK_SIZE = TILE_WIDTH;
            ^



### 不同的矩阵尺寸

In [9]:
!chmod +x compile_with_mat_size.sh && ./compile_with_mat_size.sh

Running with input size: 100
Kernel Elpased Time: 31.361 ms
Performance= 0.06 GFlop/s, Time= 31.361 msec, Size= 2000000 Ops
Computing result using host CPU...done.
Listing first 100 Differences > 0.000010...
 
  Total Errors = 0
--------------------------------

Running with input size: 200
Kernel Elpased Time: 0.120 ms
Performance= 133.29 GFlop/s, Time= 0.120 msec, Size= 16000000 Ops
Computing result using host CPU...done.
Listing first 100 Differences > 0.000010...
 
  Total Errors = 0
--------------------------------

Running with input size: 400
Kernel Elpased Time: 0.501 ms
Performance= 255.32 GFlop/s, Time= 0.501 msec, Size= 128000000 Ops
Computing result using host CPU...done.
Listing first 100 Differences > 0.000010...
 
  Total Errors = 0
--------------------------------

Running with input size: 800
Kernel Elpased Time: 4.372 ms
Performance= 234.20 GFlop/s, Time= 4.372 msec, Size= 1024000000 Ops
Computing result using host CPU...done.
Listing first 100 Differences > 0.000010.

###  不同的 TILE_SIZE

In [10]:
!bash compile_with_tile_size.sh

tile_size: 32
Kernel Elpased Time: 7.009 ms
Performance= 285.35 GFlop/s, Time= 7.009 msec, Size= 2000000000 Ops
--------------------------------

tile_size: 64
Kernel Elpased Time: 7.019 ms
Performance= 284.93 GFlop/s, Time= 7.019 msec, Size= 2000000000 Ops
--------------------------------

tile_size: 128
Kernel Elpased Time: 7.012 ms
Performance= 285.22 GFlop/s, Time= 7.012 msec, Size= 2000000000 Ops
--------------------------------

